In [5]:
import os
import json
import psycopg2
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
import asyncio
from concurrent.futures import ThreadPoolExecutor

load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

In [6]:
llm = ChatUpstage(api_key = UPSTAGE_API_KEY)

prompt_template = PromptTemplate.from_template(
    """ 
    Please do {action} on the following text:
    ---
    TEXT : {text}
    """
)

chain = prompt_template | llm | StrOutputParser()

In [3]:
# def summarize_description(text):
#     response = chain.invoke({"action": "책의 설명을 간단하게 꼭 한 문장으로 요약해줘", "text": text})
#     return response.strip()

# def summarize_sentence(text):
#     response = chain.invoke({"action": "책의 핵심문장을 심금을 울리게 꼭 한 문장으로 요약해줘", "text": text})
#     return response.strip()

# def generate_book_summaries(conn, output_json = "../data/book_summaries.json"):
#     cursor = conn.cursor()

#     cursor.execute("SELECT isbn, title, description, key_sentences From books where key_sentences IS NOT NULL;")
#     books = cursor.fetchall()

#     summaries = []

#     for isbn, title, description, key_sentences in books:
#         summarize_des = summarize_description(description)
#         summary_sen = summarize_sentence(key_sentences)

#         summaries.append({
#             "isbn": isbn,
#             "title": title,
#             "description" : summarize_des,
#             "sentence" : summary_sen
#         })
    
#     print("요약 완료!")

#     with open(output_json, "w", encoding="utf-8") as json_file:
#         json.dump(summaries, json_file, indent=4, ensure_ascii=False)

#     print("JSON 파일 저장 완료")

#     cursor.close()

In [7]:
CACHE_FILE = "../data/book_summaries_cache.json"

def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as file:
            return json.load(file)
    return {}

def save_cache(cache):
    with open(CACHE_FILE, "w", encoding="utf-8") as file:
        json.dump(cache, file, indent=4, ensure_ascii=False)

CACHE = load_cache()

def summarize_book(description, key_sentences):
    response = chain.invoke({
        "action": "책의 설명과 핵심문장을 꼭 한 문장으로 요약해줘",
        "text": f"설명: {description} / 핵심문장: {key_sentences}"
    })
    summary_list = response.strip().split("/")
    return summary_list[0].strip(), summary_list[1].strip() if len(summary_list) > 1 else ""

async def summarize_book_async(isbn, description, key_sentences):
    if isbn in CACHE:
        return isbn, CACHE[isbn]["description"], CACHE[isbn]["sentence"]
    
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        summary_des, summary_sen = await loop.run_in_executor(
            executor, summarize_book, description, key_sentences
        )
    
    CACHE[isbn] = {"description": summary_des, "sentence": summary_sen}
    save_cache(CACHE)
    return isbn, summary_des, summary_sen

async def generate_book_summaries(conn, output_json="../data/book_summaries.json"):
    cursor = conn.cursor()
    
    cursor.execute("SELECT isbn, title, description, key_sentences FROM books WHERE key_sentences IS NOT NULL LIMIT 100;")
    books = cursor.fetchall()

    tasks = [summarize_book_async(isbn, description, key_sentences) for isbn, title, description, key_sentences in books]
    results = await asyncio.gather(*tasks)

    summaries = [
        {"isbn": isbn, "description": desc, "sentence": sen}
        for isbn, desc, sen in results
    ]

    with open(output_json, "w", encoding="utf-8") as json_file:
        json.dump(summaries, json_file, indent=4, ensure_ascii=False)

    print("📄 JSON 파일 저장 완료:", output_json)
    cursor.close()

In [8]:
conn = psycopg2.connect(
    dbname="book_recommend",
    user="sesac",
    password="1234",
    host="localhost",
    port="5432"
)

# ✅ 비동기 실행
asyncio.run(generate_book_summaries(conn))

conn.close()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [11]:
import os
import json
import psycopg2
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
import time
from retry import retry

# ✅ Jupyter Notebook 환경에서 asyncio 실행 가능하도록 설정
nest_asyncio.apply()

# ✅ 환경 변수 로드
load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

if not UPSTAGE_API_KEY:
    raise ValueError("❌ ERROR: UPSTAGE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

# ✅ Upstage LLM 설정
llm = ChatUpstage(api_key=UPSTAGE_API_KEY)

prompt_template = PromptTemplate.from_template(
    """ 
    Please do {action} on the following text:
    ---
    TEXT : {text}
    """
)
chain = prompt_template | llm | StrOutputParser()

# ✅ 캐시 파일 설정
CACHE_FILE = "../data/book_summaries_cache.json"

def load_cache():
    if not os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "w", encoding="utf-8") as file:
            json.dump({}, file)  # ✅ 빈 캐시 파일 생성
    with open(CACHE_FILE, "r", encoding="utf-8") as file:
        return json.load(file)

def save_cache(cache):
    with open(CACHE_FILE, "w", encoding="utf-8") as file:
        json.dump(cache, file, indent=4, ensure_ascii=False)

CACHE = load_cache()

@retry(tries=3, delay=10)
def summarize_book(description, key_sentences):
    time.sleep(1)  # ✅ 요청 간 1초 대기 (속도 제한 방지)
    response = chain.invoke({
        "action": "책의 설명과 핵심문장을 꼭 한 문장으로 요약해줘",
        "text": f"설명: {description} / 핵심문장: {key_sentences}"
    })
    summary_list = response.strip().split("/")
    return summary_list[0].strip(), summary_list[1].strip() if len(summary_list) > 1 else ""

async def summarize_book_async(isbn, description, key_sentences):
    if isbn in CACHE:
        return isbn, CACHE[isbn]["description"], CACHE[isbn]["sentence"]
    
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        summary_des, summary_sen = await loop.run_in_executor(
            executor, summarize_book, description, key_sentences
        )
    
    CACHE[isbn] = {"description": summary_des, "sentence": summary_sen}
    save_cache(CACHE)
    return isbn, summary_des, summary_sen

async def generate_book_summaries(conn, output_json="../data/book_summaries.json"):
    cursor = conn.cursor()
    
    cursor.execute("SELECT isbn, title, description, key_sentences FROM books WHERE key_sentences IS NOT NULL LIMIT 100;")
    books = cursor.fetchall()

    tasks = [summarize_book_async(isbn, description, key_sentences) for isbn, title, description, key_sentences in books]
    results = await asyncio.gather(*tasks)

    summaries = [{"isbn": isbn, "description": desc, "sentence": sen} for isbn, desc, sen in results]

    with open(output_json, "w", encoding="utf-8") as json_file:
        json.dump(summaries, json_file, indent=4, ensure_ascii=False)

    print("📄 JSON 파일 저장 완료:", output_json)
    cursor.close()

# ✅ PostgreSQL 연결 및 실행
conn = psycopg2.connect(dbname="book_recommend", user="sesac", password="0000", host="localhost", port="5432")

try:
    asyncio.run(generate_book_summaries(conn))
finally:
    conn.close()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb8 in position 63: invalid start byte

In [10]:
!pip install retry

In [1]:
import os
import json
import psycopg2
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
import time
from retry import retry

# ✅ Jupyter Notebook 환경에서 asyncio 실행 가능하도록 설정
nest_asyncio.apply()

# ✅ 환경 변수 로드
load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

if not UPSTAGE_API_KEY:
    raise ValueError("❌ ERROR: UPSTAGE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

# ✅ Upstage LLM 설정
llm = ChatUpstage(api_key=UPSTAGE_API_KEY)

prompt_template = PromptTemplate.from_template(
    """ 
    Please do {action} on the following text:
    ---
    TEXT : {text}
    """
)
chain = prompt_template | llm | StrOutputParser()

@retry(tries=3, delay=10)
def summarize_text(action, text):
    """ Upstage API를 사용하여 텍스트를 한 문장으로 요약 """
    time.sleep(1)  # 요청 간 1초 대기 (속도 제한 방지)
    response = chain.invoke({"action": action, "text": text})
    return response.strip()

def fetch_top_10_books(conn):
    """ PostgreSQL에서 설명과 핵심 문장이 있는 상위 10개의 책 가져오기 """
    cursor = conn.cursor()
    cursor.execute("""
        SELECT isbn, title, description, key_sentences 
        FROM books 
        WHERE key_sentences IS NOT NULL 
        LIMIT 10;
    """)
    books = cursor.fetchall()
    cursor.close()
    return books

def summarize_book_data(conn):
    """ 10개의 책을 가져와서 각각 설명과 핵심 문장을 요약 """
    books = fetch_top_10_books(conn)

    print("\n📖 **10개의 책 요약 결과**\n")

    for idx, (isbn, title, description, key_sentences) in enumerate(books, start=1):
        # 설명 요약
        summarized_description = summarize_text(
            "책의 설명을 작가가 독자에게 이 책을 한마디로 표현하고자하는 말을 30글자 내외로 표현해줘", description
        )

        # 핵심 문장 요약
        summarized_sentence = summarize_text(
            "책의 핵심 문장을 작가가 독자에게 이 책을 한마디로 표현하고자하는 말을 30글자 내외로 표현해줘", key_sentences
        )

        print(f"📚 **{idx}. {title}** ({isbn})")
        print(f"   📝 설명 요약: {summarized_description}")
        print(f"   🔥 핵심 문장 요약: {summarized_sentence}\n")

# ✅ PostgreSQL 연결 및 실행
conn = psycopg2.connect(dbname="book_recommend", user="sesac", password="1234", host="localhost", port="5432",client_encoding="UTF8")

try:
    summarize_book_data(conn)  # 10개의 책 요약 실행
finally:
    conn.close()


📖 **10개의 책 요약 결과**

📚 **1. 어른의 행복은 조용하다** (139791169851053)
   📝 설명 요약: "불행에 대한 수비력을 길러주는, 조용한 행복의 발견"
   🔥 핵심 문장 요약: "오늘 실패했지만, 내일은 다시 웃을 것이다"

📚 **2. 일의 감각** (139791193383193)
   📝 설명 요약: "일 잘하는 크리에이티브의 섬세한 감각 탐구"
   🔥 핵심 문장 요약: "진정한 감각은 책임감과 몰입에서 비롯된다."

📚 **3. 내가 한 말을 내가 오해하지 않기로 함** (139791189352745)
   📝 설명 요약: "문상훈의 새로운 얼굴, 진솔한 고백과 성장"
   🔥 핵심 문장 요약: "외로움, 청춘, 시, 행복, 마음"

📚 **4. 애니만 봤더니 일본어를 잘하게 된 건에 대하여** (139791140711482)
   📝 설명 요약: "덕질로 일본어 정복, 센님의 유쾌한 일본어 학습기"
   🔥 핵심 문장 요약: 일본어, 덕질로 배우면 즐겁다!

📚 **5. 죽음 공부** (139788965966753)
   📝 설명 요약: "오늘을 충만하게 만드는 죽음의 지혜"
   🔥 핵심 문장 요약: "삶과 죽음의 경계에서, 편안한 죽음을 위해."

📚 **6. 나는 탄원한다 나를 죽이는 모든 것들에 대하여** (139791191994339)
   📝 설명 요약: "한 여배우의 진솔한 일기, 삶의 갈망과 노력"
   🔥 핵심 문장 요약: "바람에 흔들리는 삶, 진실을 갈망하다"

📚 **7. 쓸 만한 인간** (139791189856502)
   📝 설명 요약: "위로와 유머로 가득한, 솔직한 삶의 이야기"
   🔥 핵심 문장 요약: "연기, 찌질이를 위로하다. 이름, 큰 의미. 무대, 꿈의 선물."

📚 **8. 행복할 거야 이래도 되나 싶을 정도로** (139791162145043)
   📝 설명 요약: 하루를 힘차게 살아가는 당신을 위한 응원, 흔들림 없는 행복을 찾아갈 수 있도록 지지합니다.
   🔥 핵심 